# CHOLESKY Decomposition/Factorization

Given a symmetric positive definite matrix A, the Cholesky decomposition is an upper triangular matrix U (with strictly positive diagonal entries) such that:

$A=U^TU$

In [ ]:
import pycompss.interactive as ipycompss

In [ ]:
# Start PyCOMPSs runtime with graph and tracing enabled
ipycompss.start(graph=True, trace=True)

In [ ]:
from pycompss.api.task import task
from scipy import linalg
from scipy import random
import numpy as np

### Task definitions

In [ ]:
@task(returns=list)
def createBlock(BSIZE, MKLProc, diag):
    import os
    os.environ["MKL_NUM_THREADS"]=str(MKLProc)
    block = np.array(np.random.random((BSIZE, BSIZE)), dtype=np.double,copy=False)
    mb = np.matrix(block, dtype=np.double, copy=False)
    mb = mb + np.transpose(mb)
    if diag:
        mb = mb + 2*BSIZE*np.eye(BSIZE)
    return mb

@task(returns=np.ndarray)
def potrf(A, MKLProc):
    from scipy.linalg.lapack import dpotrf
    import os
    os.environ['MKL_NUM_THREADS']=str(MKLProc)
    A = dpotrf(A, lower=True)[0]
    return A

@task(returns=np.ndarray)
def solve_triangular(A, B, MKLProc):
    from scipy.linalg import solve_triangular
    from numpy import transpose
    import os
    os.environ['MKL_NUM_THREADS']=str(MKLProc)
    B = transpose(B)
    B = solve_triangular(A, B, lower=True)  # , trans='T'
    B = transpose(B)
    return B

In [ ]:
%%writefile external_module.py

from pycompss.api.task import task
import numpy as np

@task(returns=np.ndarray)
def gemm(alpha, A, B, C, beta, MKLProc):
    from scipy.linalg.blas import dgemm
    from numpy import transpose
    import os
    os.environ['MKL_NUM_THREADS']=str(MKLProc)
    B = transpose(B)
    C = dgemm(alpha, A, B, c=C, beta=beta)
    return C

### Gemm version for GPUs

In [ ]:
%%writefile -a external_module.py

from pycompss.api.implement import implement
from pycompss.api.constraint import constraint

def skcuda_matmul(alpha, A, B, C, beta):
    import os
    import pycuda.autoinit
    import pycuda.gpuarray as gpuarray
    import skcuda.linalg as culinalg
    import skcuda.cublas as cublas
    import ctypes
    ctypes.CDLL("libgomp.so.1", mode=ctypes.RTLD_GLOBAL)
    _libcusolver = ctypes.cdll.LoadLibrary("libcusolver.so")
    culinalg.init()
    from numpy import transpose
    a_gpu = gpuarray.to_gpu(A)
    B = transpose(B)
    b_gpu = gpuarray.to_gpu(B)
    c_gpu = gpuarray.to_gpu(C)
    alpha = np.float32(alpha)
    beta = np.float32(beta)
    cublas_handle = cublas.cublasCreate()
    #In this call we assume A, B, C square
    cublas.cublasDgemm(cublas_handle, "n", "n", A.shape[0],
                       B.shape[1], A.shape[1], alpha, b_gpu.gpudata, B.shape[1],
                       a_gpu.gpudata, A.shape[1], beta, c_gpu.gpudata, B.shape[0])
    cublas.cublasDestroy(cublas_handle)
    mat_res = c_gpu.get()
    return mat_res

@implement(source_class="external_module", method="gemm")
@constraint(processors=[{"ProcessorType":"CPU", "ComputingUnits":1}, {"ProcessorType":"GPU", "ComputingUnits":1}])
@task(returns=list)
def gemm_gpu(alpha, A, B, C, beta, MKLProc = 1):
    from gpu_kernels import skcuda_matmul
    res_gpu = skcuda_matmul(alpha, A, B, C, beta)
    return res_gpu

### Auxiliar functions

In [ ]:
def genMatrix(MSIZE, BSIZE, MKLProc, A):
    for i in range(MSIZE):
        A.append([])
        for j in range(MSIZE):
            A[i].append([])
    for i in range(MSIZE):
        mb = createBlock(BSIZE, MKLProc, True)
        A[i][i]=mb
        for j in range(i+1,MSIZE):
            mb = createBlock(BSIZE, MKLProc, False)
            A[i][j]=mb
            A[j][i]=mb

In [ ]:
def cholesky_blocked(MSIZE, BSIZE, mkl_threads, A):
    from external_module import gemm
    cont = 0
    for k in range(MSIZE):
        # Diagonal block factorization
        A[k][k] = potrf(A[k][k], mkl_threads)
        cont += 1
        # Triangular systems
        for i in range(k+1, MSIZE):
            A[i][k] = solve_triangular(A[k][k], A[i][k], mkl_threads)
            A[k][i] = np.zeros((BSIZE,BSIZE))
            cont += 1

        # update trailing matrix
        for i in range(k+1, MSIZE):
            for j in range(i, MSIZE):
                A[j][i] = gemm(-1.0, A[j][k], A[i][k], A[j][i], 1.0, mkl_threads)
                cont += 1
            cont += 1

    print("Number of tasks: {}".format(cont))
    return A

In [ ]:
def plot_sparsity(matrix):
    %matplotlib inline
    import matplotlib.pylab as plt
    import scipy.sparse as sps
    M = sps.csr_matrix(matrix)
    plt.spy(M)
    plt.show()
    
def plot_matrix(matrix):
    %matplotlib inline
    import matplotlib.pyplot as plt
    import numpy as np
    plt.matshow(matrix, fignum=100, cmap=plt.cm.Greys) # Greys, Blues, Purples
    plt.show()

## MAIN Code

Parameters (that can be configured in the following cell):
* MSIZE: Matrix size (default: 8)
* BSIZE: Block size (default: 1024)
* mkl_threads: Number of MKL threads (default: 1)

In [ ]:
import ipywidgets as widgets
import time
import os
from pycompss.api.api import compss_barrier
from pycompss.api.api import compss_wait_on

w_MSIZE = widgets.IntText(value=8)     # For quick demonstrations: 4
w_BSIZE = widgets.IntText(value=1024)  # For quick demonstrations: 4
w_mkl_threads = widgets.IntText(value=1)

def cholesky(MSIZE, BSIZE, mkl_threads):
    # Generate de matrix
    startTime = time.time()

    # Generate supermatrix
    A = []
    res = []
    genMatrix(MSIZE, BSIZE, mkl_threads, A)
    compss_barrier()

    initTime = time.time() - startTime
    startDecompTime = time.time()
    res = cholesky_blocked(MSIZE, BSIZE, mkl_threads, A)
    compss_barrier()

    decompTime = time.time() - startDecompTime
    totalTime = decompTime + initTime
    
    print("---------- Elapsed Times ----------")
    print("initT:{}".format(initTime))
    print("decompT:{}".format(decompTime))
    print("totalTime:{}".format(totalTime))
    print("-----------------------------------")

    # Plot the result matrix
    res = compss_wait_on(res)
    collapsed_rows_res = []
    for i in res:
        collapsed_rows_res.append(np.concatenate(i, axis=1))
    collapsed = np.concatenate(collapsed_rows_res, axis=0)
    # print(collapsed)
    plot_sparsity(collapsed)
    plot_matrix(collapsed)
    
widgets.interact_manual(cholesky, MSIZE=w_MSIZE, BSIZE=w_BSIZE, mkl_threads=w_mkl_threads)

In [ ]:
ipycompss.stop()